In [ ]:
import nltk

In [ ]:
msg = [line.strip() for line in open('smsspamcollection/SMSSpamCollection')]

In [ ]:
print(len(msg))

In [ ]:
msg[0]

In [ ]:
for i, msgs in enumerate(msg):
    print(i,msgs)
    print('\n')

In [ ]:
import pandas as pd

In [ ]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t',
                      names=['label','message'])

In [ ]:
messages

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages['length'] = messages['message'].str.len()

In [ ]:
messages['label'].value_counts()

# Removing Punctuation & Stopwords

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def text_process(mess):
    nopunc = [c for c in mess if c not in string.punctuation ]
    nopunc = ''.join(nopunc)
    return [lemmatizer.lemmatize(word) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages['message'].head(5).apply(text_process)

# Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

In [ ]:
print(len(bow_transformer.vocabulary_))

In [ ]:
messages_bow = bow_transformer.transform(messages['message'])

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
spam_detect_model = MultinomialNB().fit(messages_tfidf,messages['label'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
msg_train,msg_test,label_train,label_test = train_test_split(messages['message'],messages['label'],test_size=.3)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([('bow',CountVectorizer(analyzer=text_process)),
                    ('tfidf',TfidfTransformer()),
                    ('classifier',MultinomialNB())])

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions = pipeline.predict(msg_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(label_test,predictions))

In [ ]:
print(classification_report(label_train,msg_train))

E:\conda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\conda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\conda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\conda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score 